**Install Required Packages**


In [ ]:
# Install required packages
!pip install langchain PyMuPDF
!pip install langchain_google_genai
!pip install sentence-transformers


**Import Libraries**

In [ ]:
# Import necessary libraries
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Qdrant
import os
import numpy as np
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


**Load PDF Files**

In [ ]:
# Specify the data folder path
DATA_FOLDER_PATH = "/content/drive/MyDrive/data"

# Load PDF files from the specified folder
loader = DirectoryLoader(DATA_FOLDER_PATH, glob="**/*.pdf", loader_cls=PyMuPDFLoader, show_progress=True)
documents = loader.load()

# Split loaded documents into chunks with chunk size of 1000 characters and 40 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=40)
docs = text_splitter.split_documents(documents)


**Initialize SentenceTransformer Embedding Model**

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-Mpnet-base-v2")  # Use a model that produces 768 dimensions


**Configure Qdrant**

In [ ]:
# Qdrant Cloud Configuration
qdrant_cloud_api_key = "qdrant_api"
qdrant_url = "qdrant_url"

qdrant_cloud = Qdrant.from_documents(
    docs,
    embeddings,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_cloud_api_key,
    collection_name="new_docs",
    force_recreate=True
)


**Configure Gemini API**

In [ ]:
# Set the Gemini API key from the environment variable
gemini_api_key = 'replace_with_your_api'
if not gemini_api_key:
    raise ValueError("Gemini API key not found. Please set the GEMINI API KEY ")

genai.configure(api_key=gemini_api_key)


**Define functions**

In [ ]:
# Function to get the conversational QA chain using the Gemini model
def get_conversational_chain():
    prompt_template1 = """
    Answer the question as thoroughly as possible based on the provided context. Analyze the relationships within the content and generate meaningful, context-aware bullet points. If the answer is not available in the provided context, simply state, 'Answer is not available in the context.' Avoid providing incorrect information.
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, api_key=gemini_api_key)
    prompt = PromptTemplate(template=prompt_template1, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

# Function to interact with Gemini's generative AI
def get_gemini_response(input_text):
    # Create the Gemini model
    model = genai.GenerativeModel('gemini-pro')

    # Generate a response using the model's generate_content method
    response = model.generate_content(input_text)
    print("\n\n Summary:\n\n")
    try {
        # Access the 'candidates' list and extract the 'text' from the first candidate
        content = response.candidates[0].content.parts[0].text
        return content
    } except (AttributeError, Exception as e) {
        print(f"An error occurred: {e}")
        return "Error: Unable to retrieve response content."
    }

# Function to perform the hybrid search with RRF
def hybrid_search(query, qdrant_store, documents, k=5):
    # Perform similarity search in Qdrant
    found_docs_qdrant = qdrant_store.similarity_search(query)

    # Perform keyword search in the loaded documents
    found_docs_keyword = [doc for doc in documents if query.lower() in doc.page_content.lower()]

    # Combine results from both searches
    combined_docs = found_docs_qdrant + found_docs_keyword[:k]

    # Get ranks for reciprocal rank fusion (RRF)
    ranks = {doc.metadata['id']: idx for idx, doc in enumerate(combined_docs)}

    # Sort combined documents based on RRF
    combined_docs_sorted = sorted(combined_docs, key=lambda doc: ranks[doc.metadata['id']])

    return combined_docs_sorted[:k]
